In [1]:
"""
Reference: https://www.drivendata.co/blog/predict-pm25-benchmark/

pyhdf appears to be more powerful than gdal, so it may be worth adopting some of the 
methods used here for working with hdf files.

Additionally, the tutorial shows how to make a masked numpy array, which allows us to work
with sparse arrays? (I'm not sure how this works yet.)

Finally, the tutorial explains how to align AOD data with coordinates. This could let us
make some useful model features, like local weather conditions, etc.

"""

import os
import pandas as pd
from datetime import datetime
from osgeo import gdal
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
import keras.backend as backend
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from dateutil import parser
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC, SDS
import pyproj
from pyproj import CRS, Proj
from typing import Union
from shapely.geometry import Point, Polygon
import geopandas as gpd
from datetime import datetime, timedelta
import pickle
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score



# from pathlib import Path
# import random
# from typing import Dict, List, Union

# from cloudpathlib import S3Path
import geopandas as gpd
from netCDF4 import Dataset
# import rasterio

# DATA_PATH = Path.cwd().parent / "data"
# RAW = DATA_PATH / "raw"
# INTERIM = DATA_PATH / "interim"

In [2]:
"""
DATA PROCESSING

"""

'\nDATA PROCESSING\n\n'

In [3]:
# Loop over orbits to apply the attributes
def calibrate_data(dataset: SDS, shape: list[int], calibration_dict: dict):
    """Given a MAIAC dataset and calibration parameters, return a masked
    array of calibrated data.
    
    Args:
        dataset (SDS): dataset in SDS format (e.g. blue band AOD).
        shape (List[int]): dataset shape as a list of [orbits, height, width].
        calibration_dict (Dict): dictionary containing, at a minimum,
            `valid_range` (list or tuple), `_FillValue` (int or float),
            `add_offset` (float), and `scale_factor` (float).
    
    Returns:
        corrected_AOD (np.ma.MaskedArray): masked array of calibrated data
            with a fill value of nan.
    """
    corrected_AOD = np.ma.empty(shape, dtype=np.double)
    for orbit in range(shape[0]):
        data = dataset[orbit, :, :].astype(np.double)
        invalid_condition = (
            (data < calibration_dict["valid_range"][0]) |
            (data > calibration_dict["valid_range"][1]) |
            (data == calibration_dict["_FillValue"])
        )
        data[invalid_condition] = np.nan
        data = (
            (data - calibration_dict["add_offset"]) *
            calibration_dict["scale_factor"]
        )
        data = np.ma.masked_array(data, np.isnan(data))
        corrected_AOD[orbit, : :] = data
    corrected_AOD.fill_value = np.nan
    return corrected_AOD



In [4]:
"""

Aligning AOD data with real world coordinates


"""


def create_meshgrid(alignment_dict: dict, shape: list[int]):
    """Given an image shape, create a meshgrid of points
    between bounding coordinates.
    
    Args:
        alignment_dict (Dict): dictionary containing, at a minimum,
            `upper_left` (tuple), `lower_right` (tuple), `crs` (str),
            and `crs_params` (tuple).
        shape (List[int]): dataset shape as a list of
            [orbits, height, width].
    
    Returns:
        xv (np.array): x (longitude) coordinates.
        yv (np.array): y (latitude) coordinates.
    """
    # Determine grid bounds using two coordinates
    x0, y0 = alignment_dict["upper_left"]
    x1, y1 = alignment_dict["lower_right"]
    
    # Interpolate points between corners, inclusive of bounds
    x = np.linspace(x0, x1, shape[2], endpoint=True)
    y = np.linspace(y0, y1, shape[1], endpoint=True)
    
    # Return two 2D arrays representing X & Y coordinates of all points
    xv, yv = np.meshgrid(x, y)
    return xv, yv

In [5]:


# Source: https://spatialreference.org/ref/sr-org/modis-sinusoidal/proj4js/

def transform_arrays(
    xv: Union[np.array, float],
    yv: Union[np.array, float],
    crs_from: CRS,
    crs_to: CRS
):
    """Transform points or arrays from one CRS to another CRS.
    
    Args:
        xv (np.array or float): x (longitude) coordinates or value.
        yv (np.array or float): y (latitude) coordinates or value.
        crs_from (CRS): source coordinate reference system.
        crs_to (CRS): destination coordinate reference system.
    
    Returns:
        lon, lat (tuple): x coordinate(s), y coordinate(s)
    """
    transformer = pyproj.Transformer.from_crs(
        crs_from,
        crs_to,
        always_xy=True,
    )
    
    lon, lat = transformer.transform(xv, yv)
    return lon, lat



# Project sinu grid onto wgs84 grid

In [6]:
#Currently not used
def convert_array_to_df(
    corrected_arr: np.ma.MaskedArray,
    lat:np.ndarray,
    lon: np.ndarray,
    granule_id: str,
    crs: CRS,
    total_bounds: np.ndarray = None
):
    """Align data values with latitude and longitude coordinates
    and return a GeoDataFrame.
    
    Args:
        corrected_arr (np.ma.MaskedArray): data values for each pixel.
        lat (np.ndarray): latitude for each pixel.
        lon (np.ndarray): longitude for each pixel.
        granule_id (str): granule name.
        crs (CRS): coordinate reference system
        total_bounds (np.ndarray, optional): If provided,
            will filter out points that fall outside of these bounds.
            Composed of xmin, ymin, xmax, ymax.
    """
    lats = lat.ravel()
    lons = lon.ravel()
    n_orbits = len(corrected_arr)
    size = lats.size
    values = {
        "value": np.concatenate([d.data.ravel() for d in corrected_arr]),
        "lat": np.tile(lats, n_orbits),
        "lon": np.tile(lons, n_orbits),
        "orbit": np.arange(n_orbits).repeat(size),
        "granule_id": [granule_id] * size * n_orbits
        
    }
    
    df = pd.DataFrame(values).dropna()
    if total_bounds is not None:
        x_min, y_min, x_max, y_max = total_bounds
        df = df[df.lon.between(x_min, x_max) & df.lat.between(y_min, y_max)]
    
    gdf = gpd.GeoDataFrame(df)
    gdf["geometry"] = gpd.points_from_xy(gdf.lon, gdf.lat)
    gdf.crs = crs
    return gdf[["granule_id", "orbit", "geometry", "value"]].reset_index(drop=True)

In [7]:
"""

Some more helpful functions from the tutorial


"""

def create_calibration_dict(data: SDS):
    """Define calibration dictionary given a SDS dataset,
    which contains:
        - name
        - scale factor
        - offset
        - unit
        - fill value
        - valid range
    
    Args:
        data (SDS): dataset in the SDS format.
    
    Returns:
        calibration_dict (Dict): dict of calibration parameters.
    """
    return data.attributes()


def create_alignment_dict(hdf: SD):
    """Define alignment dictionary given a SD data file, 
    which contains:
        - upper left coordinates
        - lower right coordinates
        - coordinate reference system (CRS)
        - CRS parameters
    
    Args:
        hdf (SD): hdf data object
    
    Returns:
        alignment_dict (Dict): dict of alignment parameters.
    """
    group_1 = hdf.attributes()["StructMetadata.0"].split("END_GROUP=GRID_1")[0]
    hdf_metadata = dict([x.split("=") for x in group_1.split() if "=" in x])
    alignment_dict = {
        "upper_left": eval(hdf_metadata["UpperLeftPointMtrs"]),
        "lower_right": eval(hdf_metadata["LowerRightMtrs"]),
        "crs": hdf_metadata["Projection"],
        "crs_params": eval(hdf_metadata["ProjParams"])
    }
    
    return alignment_dict

In [28]:
from shapely.geometry import Point, Polygon

"""

Everything here is original code that uses the functions from the tutorial.

within(): taken from https://automating-gis-processes.github.io/2017/lessons/L3/point-in-polygon.html

Make_Submatrix(): A function which takes raw AOD matrix and a Grid ID of interest as input and outputs a submatrix 
of AOD values which are inside this grid point (5km by 5km).

Currently, Make_Submatrix() only returns the number of pixels in the AOD matrix are within the location determined
by Grid ID.

The rest of the code in this cell runs extremely slowly, but this is because we are running it for all possible 
combinations of HDF file and Grid ID. When we actually use these functions to run a model on a given Grid ID and 
datetime, we will first filter the set of HDF files such that we only search through HDF files with matching city 
and matching datetime.

"""


#Helper function
def Make_Poly(polyString):
    poly_coords = []
    for string in polyString.split(','):
        split_string = string.split(' ')
        if split_string[0] == 'POLYGON':
            split_string = split_string[1:]
            split_string[0] = str(split_string[0])[2:]
    #         print(tuple(float(x) for x in split_string))
        elif split_string[0] == '':
            split_string = split_string[1:]
        if split_string[1][-2] == ')':
            split_string[1] = split_string[1][0:-2]
        poly_coords.append(tuple(float(x) for x in split_string))

    return Polygon(poly_coords)



#Main function
def Make_Submatrix(corrected_AOD, lon, lat, alignment_dict, grid_md, gridID, is_hdf):
    
    poly = Make_Poly(grid_md['wkt'][gridID])
    if is_hdf:        
        return_list = []
        for band in range(len(corrected_AOD)):
            counter = 0
            triples_array = []
            for i in range(len(corrected_AOD[0])):
                if lat[i,0] > poly.bounds[3]:
                    continue

                if lat[i,0] < poly.bounds[1]:
                    continue

                for j in range(len(corrected_AOD[0][0])):
                    if lon[i,j] > poly.bounds[2]:
                        continue
                    p1 = Point(lon[i,j], lat[i,j]) 
                    if(p1.within(poly)):
                        triples_array.append((i, j, corrected_AOD[band,i,j]))


            if len(triples_array) == 0:
                continue

            temp_array = np.zeros((10,10))
            temp_array = np.ma.masked_array(temp_array, mask=np.ones((10,10)))

            i_array = [x[0] for x in triples_array]
            j_array = [x[1] for x in triples_array]
            min_i = min(i_array)
            min_j = min(j_array)



            for triple in triples_array:
                if triple[2] is np.ma.masked:
                    pass

                else:
                    temp_array[triple[0]-min_i, triple[1]-min_j] = triple[2]
            return_list.append(temp_array)
        
    #this case runs when we are working with a .nc file    
    else:
        
    #Since the lon, lat arrays are masked, we will use the fact that AOD arrays from the .nc file have a constant
    #pattern (the valid regions form a curve going from the top right to the bottom left) to narrow down the 
    #search
        return_list = []
        triples_array = []
        for i in range(len(corrected_AOD)):
            min_unmasked_index = np.ma.flatnotmasked_edges(lat[i])[0]
            max_unmasked_index = np.ma.flatnotmasked_edges(lat[i])[1]
            if lat[i][min_unmasked_index] < poly.bounds[1]:
                continue
            if lat[i][max_unmasked_index] > poly.bounds[3]:
                continue
            for j in range(len(corrected_AOD[0])):
                
                #need to check if our longitude, latitude values are masked
                if lon[i,j] is np.ma.masked or lat[i,j] is np.ma.masked:
                    pass
                
                p1 = Point(lon[i,j], lat[i,j]) 
                if(p1.within(poly)):
                    triples_array.append((i, j, corrected_AOD[i,j]))
                    
                    
            if len(triples_array) == 0:
                continue

            temp_array = np.zeros((10,10))
            temp_array = np.ma.masked_array(temp_array, mask=np.ones((10,10)))

            i_array = [x[0] for x in triples_array]
            j_array = [x[1] for x in triples_array]
            min_i = min(i_array)
            min_j = min(j_array)



            for triple in triples_array:
                if triple[2] is np.ma.masked:
                    pass

                else:
                    temp_array[triple[0]-min_i, triple[1]-min_j] = triple[2]
            return_list.append(temp_array)
        print(temp_array)
                
            
        
        
        
        
        
    return return_list 
    


In [9]:
#Note that each AOD_array in array_of_AOD_arrays should be a precalculated subarray corresponding to grid id
#Currently not using the is_hdf parameter but will probably have to as we make this more sophisticated
def collect_features(array_of_AOD_arrays, area_per_subarray, is_hdf):
    

    total_values = len(array_of_AOD_arrays)*area_per_subarray

    all_values = np.zeros((total_values))
    all_values = np.ma.masked_array(all_values, mask=np.ones((total_values)))


    counter = 0
    for AOD_array in array_of_AOD_arrays:
        for row in AOD_array:
            for value in row:
                if not (value is np.ma.masked):
                    all_values[counter] = value
                    counter+=1


    mean = np.ma.mean(all_values)
    minimum = np.ma.min(all_values)
    maximum = np.ma.max(all_values)
    std = np.ma.std(all_values)
    summ = np.ma.sum(all_values)


    
    
    
    return(mean, minimum, maximum, std, summ)

In [68]:
import random

def get_features(train_labels, satellite_metadata, grid_id_list, training):
    features = []
    

    r = list(range(len(train_labels)))
#     random.shuffle(r)
    for i in r:
        
        print(i)
#         print(features)

        satellite_metadata_cut = satellite_metadata
#         satellite_metadata = satellite_metadata[satellite_metadata.granule_id.str.endswith('f')] #remove this
        grid_id = train_labels['grid_id'][i]
        j = grid_id_list.index(grid_id)
        location = grid_metadata['location'][j]
        tz = grid_metadata['tz'][j]
        datetime = pd.to_datetime(
            train_labels['datetime'][j],
            format="%Y%m%dT%H:%M:%S",
            utc=True
        )
        polygon = grid_metadata['wkt'][j]

        if location == 'Delhi':
            satellite_metadata_cut = satellite_metadata[satellite_metadata['location'] == 'dl'].copy()
        elif location == 'Los Angeles (SoCAB)':
            satellite_metadata_cut = satellite_metadata[satellite_metadata['location'] == 'la'].copy()
        elif location == 'Taipei':
            satellite_metadata_cut = satellite_metadata[satellite_metadata['location'] == 'tpe'].copy()


        valid_datetime = [None]*len(satellite_metadata_cut)
        satellite_metadata_cut.reset_index(drop=True, inplace=True) # ensure indexes pair with number of rows
    #     count = 0
        for index, row in satellite_metadata_cut.iterrows():
            


            datetime1 = pd.to_datetime((row['time_start']), format="%Y%m%dT%H:%M:%S", 
                                    utc=True) 
            datetime2 = pd.to_datetime(row['time_end'], format="%Y%m%dT%H:%M:%S", 
                                                utc=True) 


            truth1 = (datetime <= datetime2)
            truth2 = datetime2 <= datetime + timedelta(hours=24)
 
            valid_datetime[index] = (truth1 & truth2)



        satellite_metadata_cut['valid_datetime'] = valid_datetime
        satellite_metadata_cut = satellite_metadata_cut[satellite_metadata_cut['valid_datetime'] == True]

        raw_hdf_nc_set = list(satellite_metadata_cut['granule_id']) 


        print(raw_hdf_nc_set, location)
#         print('next:')

        list_of_all_AOD_arrays = []
        for hdf_nc_filename in raw_hdf_nc_set:
            
#             print('now working with:', hdf_nc_filename)
            
            if training:
                filepath = 'train/' + hdf_nc_filename
            else:
                filepath = 'test/' + hdf_nc_filename

#             print('here:', hdf_nc_filename)
            if hdf_nc_filename.endswith('f'):
                
#                 print('this should happen')
            
                raw_hdf = SD(filepath)

                alignment_dict = create_alignment_dict(raw_hdf)

                blue_band_AOD = raw_hdf.select("Optical_Depth_047")
                name, num_dim, shape, types, num_attr = blue_band_AOD.info()
                calibration_dict = create_calibration_dict(blue_band_AOD)
                corrected_AOD = calibrate_data(blue_band_AOD, shape, calibration_dict)

                xv, yv = create_meshgrid(alignment_dict, shape)            
                sinu_crs = Proj(f"+proj=sinu +R={alignment_dict['crs_params'][0]} +nadgrids=@null +wktext").crs
                wgs84_crs = CRS.from_epsg("4326")            
                lon, lat = transform_arrays(xv, yv, sinu_crs, wgs84_crs)

                temp = Make_Submatrix(corrected_AOD, lon, lat, alignment_dict, grid_metadata, grid_id, True)
                if len(temp) == 0:
                    continue
                for AOD_array in temp:
                    list_of_all_AOD_arrays.append(AOD_array)
                    
            else:
#                 continue
                fh = Dataset(filepath, mode='r')
                corrected_AOD = (fh.groups['4.4_KM_PRODUCTS']['AUXILIARY']['Aerosol_Optical_Depth_Raw'])
                lat = (fh['4.4_KM_PRODUCTS']['Latitude'])
                lon = (fh['4.4_KM_PRODUCTS']['Longitude'])
                alignment_dict = None
                temp = Make_Submatrix(corrected_AOD, lon, lat, alignment_dict, grid_metadata, grid_id, False)
                fh.close()
                if len(temp) == 0:
                    continue
                for AOD_array in temp:
                    list_of_all_AOD_arrays.append(AOD_array)
                
                    
                
                
                

        if list_of_all_AOD_arrays == 0:
            if training:
                features.append(np.array((np.nan, np.nan, np.nan, np.nan, np.nan)))
            else:
                features.append(np.array((0, 0, 0, 0, 0)))
        else:
#             print(list_of_all_AOD_arrays)
            features.append(np.array(collect_features(list_of_all_AOD_arrays, 100, True)))
#             print(features)

    print(features)
    return features

In [69]:
#Getting data to train the model
from datetime import datetime, timedelta


# features = []
train_labels = pd.read_csv("train_labels_100.csv") # Smallest subset
grid_metadata = pd.read_csv("grid_metadata.csv", index_col=0)
satellite_metadata = pd.read_csv("pm25_satellite_metadata.csv")
# satellite_metadata = satellite_metadata[satellite_metadata.granule_id.str.endswith('f')] #We now want .nc files too
satellite_metadata = satellite_metadata[satellite_metadata['split'] == 'train'].copy()


# print(grid_metadata['tz'].keys())
grid_id_list = list(grid_metadata['tz'].keys())
features = get_features(train_labels, satellite_metadata, grid_id_list, True)
    

0
['20180201T191000_maiac_la_0.hdf'] Los Angeles (SoCAB)
1
['20180201T191000_maiac_la_0.hdf'] Los Angeles (SoCAB)
2
['20180201T191000_maiac_la_0.hdf'] Los Angeles (SoCAB)
3
['20180201T191000_maiac_la_0.hdf'] Los Angeles (SoCAB)
4
['20180201T191000_maiac_la_0.hdf'] Los Angeles (SoCAB)
5
['20180201T191000_maiac_la_0.hdf'] Los Angeles (SoCAB)
6
['20180202T195000_maiac_la_0.hdf'] Los Angeles (SoCAB)
7
['20180202T195000_maiac_la_0.hdf'] Los Angeles (SoCAB)
8
['20180202T195000_maiac_la_0.hdf'] Los Angeles (SoCAB)
9
['20180202T195000_maiac_la_0.hdf'] Los Angeles (SoCAB)
10
['20180203T203000_maiac_la_0.hdf', '20180203T193400_misr_la_0.nc'] Los Angeles (SoCAB)


/Users/johnblackwelder/miniforge3/envs/nasa_conda/lib/python3.10/site-packages/shapely/geometry/point.py:262: UserWarning: Warning: converting a masked element to nan.
  dx = c_double(coords[0])
/Users/johnblackwelder/miniforge3/envs/nasa_conda/lib/python3.10/site-packages/shapely/geometry/point.py:263: UserWarning: Warning: converting a masked element to nan.
  dy = c_double(coords[1])


[[0.02452925406396389 -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]]
11
['20180203T203000_maiac_la_0.hdf', '20180203T193400_misr_la_0.nc'] Los Angeles (SoCAB)


/Users/johnblackwelder/miniforge3/envs/nasa_conda/lib/python3.10/site-packages/shapely/geometry/point.py:262: UserWarning: Warning: converting a masked element to nan.
  dx = c_double(coords[0])
/Users/johnblackwelder/miniforge3/envs/nasa_conda/lib/python3.10/site-packages/shapely/geometry/point.py:263: UserWarning: Warning: converting a masked element to nan.
  dy = c_double(coords[1])


[[-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]]
12
['20180202T032500_maiac_tpe_0.hdf', '20180202T032500_maiac_tpe_1.hdf'] Taipei


...

96
['20180203T041000_maiac_tpe_0.hdf', '20180203T023000_maiac_tpe_0.hdf'] Taipei


/var/folders/s9/bqp3gwh57sv7rgc2_ynsbcdm0000gn/T/ipykernel_5170/3555266630.py:125: UserWarning: Warning: converting a masked element to nan.
  features.append(np.array(collect_features(list_of_all_AOD_arrays, 100, True)))


97
['20180204T031500_maiac_tpe_0.hdf', '20180204T031500_maiac_tpe_1.hdf'] Taipei


/var/folders/s9/bqp3gwh57sv7rgc2_ynsbcdm0000gn/T/ipykernel_5170/3555266630.py:125: UserWarning: Warning: converting a masked element to nan.
  features.append(np.array(collect_features(list_of_all_AOD_arrays, 100, True)))


98
['20180202T064500_maiac_dl_0.hdf'] Delhi
99
['20180202T064500_maiac_dl_0.hdf'] Delhi
[array([0.08363158, 0.063     , 0.107     , 0.01132631, 3.178     ]), array([0.08105263, 0.059     , 0.106     , 0.01330205, 3.08      ]), array([0.07797297, 0.036     , 0.133     , 0.03143289, 2.885     ]), array([0.10215385, 0.052     , 0.161     , 0.02714778, 3.984     ]), array([0.12309091, 0.093     , 0.163     , 0.02256991, 5.416     ]), array([0.13273913, 0.109     , 0.166     , 0.01513768, 3.053     ]), array([0.12613158, 0.077     , 0.192     , 0.03005361, 4.793     ]), array([0.099     , 0.025     , 0.134     , 0.02732947, 1.98      ]), array([0.223     , 0.139     , 0.46      , 0.09415147, 3.568     ]), array([0.17009091, 0.096     , 0.226     , 0.03558972, 7.484     ]), array([0.07597805, 0.02452925, 0.124     , 0.03351969, 3.8748803 ]), array([0.0768    , 0.        , 0.139     , 0.03463081, 2.304     ]), array([nan, nan, nan, nan, nan]), array([nan, nan, nan, nan, nan]), array([nan, nan

In [ ]:
import pickle
    
pickle.dump( features, open( "save1.p", "wb" ) )
print(features)



In [72]:
#Making the model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
import tensorflow as tf



features_cut = features.copy()

        


labels_array = np.array(train_labels.value)
cut_labels_array = list(labels_array[0:227].copy())

i = 0
while( i < len(features_cut)):
    print(features_cut[i][0])
    if np.isnan(features_cut[i][0]):
        features_cut.pop(i)
        cut_labels_array.pop(i)
    else:
        i+=1
        
print(len(features_cut))
print(len(cut_labels_array))

def make_model():

    model = Sequential()
    model.add(Dense(13, input_dim=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

model = make_model()
model.summary()


# estimator = KerasRegressor(build_fn=make_model, nb_epoch=100, batch_size=5, verbose=0)

X = np.array(features_cut)
Y = np.array(cut_labels_array)
model.fit(X, Y, epochs=1000, batch_size=10)

# kfold = KFold(n_splits=10)
# results = cross_val_score(estimator, X, Y, n_jobs=1)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))


0.08363157894736843
0.08105263157894738
0.07797297297297297
0.10215384615384615
0.1230909090909091
0.1327391304347826
0.1261315789473684
0.099
0.22300000000000003
0.1700909090909091
0.07597804515355942
0.07680000000000001
nan
nan
nan
nan
nan
nan
nan
0.8204347826086957
0.6804117647058824
0.7850000000000001
0.5046904761904761
0.7068333333333333
0.572375
0.5551489361702128
0.688741935483871
0.746741935483871
0.31638633258292015
0.38795496642911753
0.2993186286834248
nan
nan
nan
nan
nan
nan
nan
0.8204347826086957
0.6804117647058824
0.7850000000000001
0.5046904761904761
0.7068333333333333
0.5551489361702128
0.688741935483871
0.746741935483871
0.31638633258292015
0.38795496642911753
0.2993186286834248
0.08363157894736843
0.08105263157894738
0.07797297297297297
0.10215384615384615
0.1230909090909091
0.1327391304347826
0.1261315789473684
0.099
0.22300000000000003
0.1700909090909091
0.07597804515355942
0.07680000000000001
nan
nan
nan
nan
nan
nan
nan
0.8204347826086957
0.6804117647058824
0.78500

8/8 [==============================] - 0s 594us/step - loss: 882.3279
Epoch 71/1000
8/8 [==============================] - 0s 512us/step - loss: 866.0236
Epoch 72/1000
8/8 [==============================] - 0s 598us/step - loss: 853.0408
Epoch 73/1000
8/8 [==============================] - 0s 590us/step - loss: 847.6057
Epoch 74/1000
8/8 [==============================] - 0s 471us/step - loss: 841.9926
Epoch 75/1000
8/8 [==============================] - 0s 542us/step - loss: 840.1575
Epoch 76/1000
8/8 [==============================] - 0s 470us/step - loss: 833.6253
Epoch 77/1000
8/8 [==============================] - 0s 544us/step - loss: 831.4041
Epoch 78/1000
8/8 [==============================] - 0s 528us/step - loss: 830.3943
Epoch 79/1000
8/8 [==============================] - 0s 494us/step - loss: 831.1483
Epoch 80/1000
8/8 [==============================] - 0s 487us/step - loss: 827.9781
Epoch 81/1000
8/8 [==============================] - 0s 526us/step - loss: 827.6030
Epoch 

8/8 [==============================] - 0s 509us/step - loss: 812.2966
Epoch 167/1000
8/8 [==============================] - 0s 616us/step - loss: 812.4250
Epoch 168/1000
8/8 [==============================] - 0s 551us/step - loss: 812.9915
Epoch 169/1000
8/8 [==============================] - 0s 423us/step - loss: 812.2469
Epoch 170/1000
8/8 [==============================] - 0s 621us/step - loss: 811.7849
Epoch 171/1000
8/8 [==============================] - 0s 539us/step - loss: 811.1484
Epoch 172/1000
8/8 [==============================] - 0s 528us/step - loss: 811.2190
Epoch 173/1000
8/8 [==============================] - 0s 487us/step - loss: 810.9323
Epoch 174/1000
8/8 [==============================] - 0s 529us/step - loss: 811.1883
Epoch 175/1000
8/8 [==============================] - 0s 479us/step - loss: 811.0790
Epoch 176/1000
8/8 [==============================] - 0s 491us/step - loss: 810.7376
Epoch 177/1000
8/8 [==============================] - 0s 495us/step - loss: 811.

8/8 [==============================] - 0s 466us/step - loss: 795.9629
Epoch 263/1000
8/8 [==============================] - 0s 414us/step - loss: 796.0364
Epoch 264/1000
8/8 [==============================] - 0s 487us/step - loss: 796.6215
Epoch 265/1000
8/8 [==============================] - 0s 484us/step - loss: 796.1309
Epoch 266/1000
8/8 [==============================] - 0s 470us/step - loss: 795.4854
Epoch 267/1000
8/8 [==============================] - 0s 457us/step - loss: 795.3528
Epoch 268/1000
8/8 [==============================] - 0s 482us/step - loss: 795.9804
Epoch 269/1000
8/8 [==============================] - 0s 511us/step - loss: 795.9158
Epoch 270/1000
8/8 [==============================] - 0s 411us/step - loss: 794.9524
Epoch 271/1000
8/8 [==============================] - 0s 472us/step - loss: 794.8494
Epoch 272/1000
8/8 [==============================] - 0s 471us/step - loss: 795.0363
Epoch 273/1000
8/8 [==============================] - 0s 438us/step - loss: 794.

8/8 [==============================] - 0s 430us/step - loss: 782.2750
Epoch 359/1000
8/8 [==============================] - 0s 509us/step - loss: 781.3564
Epoch 360/1000
8/8 [==============================] - 0s 447us/step - loss: 783.0368
Epoch 361/1000
8/8 [==============================] - 0s 486us/step - loss: 779.8352
Epoch 362/1000
8/8 [==============================] - 0s 460us/step - loss: 779.3674
Epoch 363/1000
8/8 [==============================] - 0s 477us/step - loss: 779.9333
Epoch 364/1000
8/8 [==============================] - 0s 441us/step - loss: 782.1061
Epoch 365/1000
8/8 [==============================] - 0s 488us/step - loss: 779.2969
Epoch 366/1000
8/8 [==============================] - 0s 474us/step - loss: 779.5123
Epoch 367/1000
8/8 [==============================] - 0s 454us/step - loss: 779.4724
Epoch 368/1000
8/8 [==============================] - 0s 470us/step - loss: 778.8676
Epoch 369/1000
8/8 [==============================] - 0s 484us/step - loss: 779.

...

8/8 [==============================] - 0s 501us/step - loss: 746.7902
Epoch 551/1000
8/8 [==============================] - 0s 431us/step - loss: 746.5654
Epoch 552/1000
8/8 [==============================] - 0s 498us/step - loss: 748.6683
Epoch 553/1000
8/8 [==============================] - 0s 468us/step - loss: 747.6310
Epoch 554/1000
8/8 [==============================] - 0s 496us/step - loss: 745.7017
Epoch 555/1000
8/8 [==============================] - 0s 497us/step - loss: 745.1723
Epoch 556/1000
8/8 [==============================] - 0s 522us/step - loss: 745.7584
Epoch 557/1000
8/8 [==============================] - 0s 485us/step - loss: 746.8953
Epoch 558/1000
8/8 [==============================] - 0s 441us/step - loss: 746.5891
Epoch 559/1000
8/8 [==============================] - 0s 504us/step - loss: 745.6487
Epoch 560/1000
8/8 [==============================] - 0s 447us/step - loss: 748.2737
Epoch 561/1000
8/8 [==============================] - 0s 505us/step - loss: 745.

8/8 [==============================] - 0s 442us/step - loss: 730.0143
Epoch 647/1000
8/8 [==============================] - 0s 476us/step - loss: 730.2430
Epoch 648/1000
8/8 [==============================] - 0s 519us/step - loss: 730.4903
Epoch 649/1000
8/8 [==============================] - 0s 504us/step - loss: 728.4931
Epoch 650/1000
8/8 [==============================] - 0s 452us/step - loss: 729.5411
Epoch 651/1000
8/8 [==============================] - 0s 498us/step - loss: 728.8691
Epoch 652/1000
8/8 [==============================] - 0s 461us/step - loss: 729.2442
Epoch 653/1000
8/8 [==============================] - 0s 479us/step - loss: 729.0516
Epoch 654/1000
8/8 [==============================] - 0s 467us/step - loss: 728.5324
Epoch 655/1000
8/8 [==============================] - 0s 476us/step - loss: 728.1936
Epoch 656/1000
8/8 [==============================] - 0s 446us/step - loss: 728.4000
Epoch 657/1000
8/8 [==============================] - 0s 463us/step - loss: 728.

8/8 [==============================] - 0s 508us/step - loss: 713.3629
Epoch 743/1000
8/8 [==============================] - 0s 454us/step - loss: 712.4847
Epoch 744/1000
8/8 [==============================] - 0s 481us/step - loss: 712.7214
Epoch 745/1000
8/8 [==============================] - 0s 497us/step - loss: 712.8171
Epoch 746/1000
8/8 [==============================] - 0s 485us/step - loss: 712.9419
Epoch 747/1000
8/8 [==============================] - 0s 480us/step - loss: 713.2050
Epoch 748/1000
8/8 [==============================] - 0s 432us/step - loss: 715.3544
Epoch 749/1000
8/8 [==============================] - 0s 447us/step - loss: 712.1019
Epoch 750/1000
8/8 [==============================] - 0s 488us/step - loss: 711.8725
Epoch 751/1000
8/8 [==============================] - 0s 477us/step - loss: 712.3389
Epoch 752/1000
8/8 [==============================] - 0s 494us/step - loss: 712.0717
Epoch 753/1000
8/8 [==============================] - 0s 523us/step - loss: 711.

8/8 [==============================] - 0s 479us/step - loss: 695.9379
Epoch 839/1000
8/8 [==============================] - 0s 463us/step - loss: 696.1255
Epoch 840/1000
8/8 [==============================] - 0s 479us/step - loss: 695.7787
Epoch 841/1000
8/8 [==============================] - 0s 467us/step - loss: 695.7700
Epoch 842/1000
8/8 [==============================] - 0s 468us/step - loss: 695.5937
Epoch 843/1000
8/8 [==============================] - 0s 521us/step - loss: 693.9307
Epoch 844/1000
8/8 [==============================] - 0s 455us/step - loss: 694.3625
Epoch 845/1000
8/8 [==============================] - 0s 537us/step - loss: 694.5692
Epoch 846/1000
8/8 [==============================] - 0s 462us/step - loss: 694.0663
Epoch 847/1000
8/8 [==============================] - 0s 484us/step - loss: 694.7670
Epoch 848/1000
8/8 [==============================] - 0s 437us/step - loss: 694.2490
Epoch 849/1000
8/8 [==============================] - 0s 450us/step - loss: 694.

8/8 [==============================] - 0s 466us/step - loss: 680.2996
Epoch 935/1000
8/8 [==============================] - 0s 427us/step - loss: 679.2195
Epoch 936/1000
8/8 [==============================] - 0s 468us/step - loss: 678.8291
Epoch 937/1000
8/8 [==============================] - 0s 528us/step - loss: 678.5684
Epoch 938/1000
8/8 [==============================] - 0s 460us/step - loss: 677.6439
Epoch 939/1000
8/8 [==============================] - 0s 506us/step - loss: 677.0715
Epoch 940/1000
8/8 [==============================] - 0s 478us/step - loss: 677.8385
Epoch 941/1000
8/8 [==============================] - 0s 504us/step - loss: 677.2523
Epoch 942/1000
8/8 [==============================] - 0s 487us/step - loss: 676.3016
Epoch 943/1000
8/8 [==============================] - 0s 495us/step - loss: 677.3034
Epoch 944/1000
8/8 [==============================] - 0s 496us/step - loss: 676.0341
Epoch 945/1000
8/8 [==============================] - 0s 519us/step - loss: 676.

In [73]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

predicted = model.predict(np.array(features_cut))
print(mean_squared_error(predicted, Y))
r2_score(predicted, Y)


665.5902411374832


0.7057558285083234

In [ ]:
pickle.dump( model, open( "model1.p", "wb" ) )

In [41]:
#testing the model, first getting features

features = []

test_labels = pd.read_csv("submission_format.csv") # Smallest subset
grid_metadata = pd.read_csv("grid_metadata.csv", index_col=0)
satellite_metadata = pd.read_csv("pm25_satellite_metadata.csv")
satellite_metadata = satellite_metadata[satellite_metadata.granule_id.str.endswith('f')]
satellite_metadata = satellite_metadata[satellite_metadata['split'] == 'test'].copy()


grid_id_list = list(grid_metadata['wkt'].keys())

features_test = get_features(test_labels, satellite_metadata, grid_id_list, False) 

7915
['20170109T062500_maiac_dl_0.hdf'] Delhi
1849
['20170110T201500_maiac_la_0.hdf'] Los Angeles (SoCAB)


/var/folders/s9/bqp3gwh57sv7rgc2_ynsbcdm0000gn/T/ipykernel_5170/4213421721.py:124: UserWarning: Warning: converting a masked element to nan.
  features.append(np.array(collect_features(list_of_all_AOD_arrays, 100, True)))


8991
['20170111T025500_maiac_tpe_0.hdf', '20170111T025500_maiac_tpe_1.hdf'] Taipei


/var/folders/s9/bqp3gwh57sv7rgc2_ynsbcdm0000gn/T/ipykernel_5170/4213421721.py:124: UserWarning: Warning: converting a masked element to nan.
  features.append(np.array(collect_features(list_of_all_AOD_arrays, 100, True)))


8064
['20170109T062500_maiac_dl_0.hdf'] Delhi
8554
['20170109T030500_maiac_tpe_0.hdf', '20170109T030500_maiac_tpe_1.hdf'] Taipei


...

/var/folders/s9/bqp3gwh57sv7rgc2_ynsbcdm0000gn/T/ipykernel_5170/4213421721.py:124: UserWarning: Warning: converting a masked element to nan.
  features.append(np.array(collect_features(list_of_all_AOD_arrays, 100, True)))


6221
['20170109T062500_maiac_dl_0.hdf'] Delhi
2398
['20170109T193000_maiac_la_0.hdf'] Los Angeles (SoCAB)


/var/folders/s9/bqp3gwh57sv7rgc2_ynsbcdm0000gn/T/ipykernel_5170/4213421721.py:124: UserWarning: Warning: converting a masked element to nan.
  features.append(np.array(collect_features(list_of_all_AOD_arrays, 100, True)))


4066
['20170110T201500_maiac_la_0.hdf'] Los Angeles (SoCAB)


KeyboardInterrupt: 

In [ ]:
hdf = SD('test/20170109T193000_maiac_la_0.hdf')
hdf47 = hdf.select(0)
for i in range(4):
    plt.imshow(hdf47.get()[i])
    plt.show()
print(hdf47.get()[0][1199, 1199])